# ProyectoDSParteI_Hamburg


## Abstract

Este proyecto realiza un Análisis Exploratorio de Datos (EDA) sobre canciones populares en Spotify (“Top Hits 2000–2019”), utilizando un archivo **`songs_normalize.csv`** ubicado en la misma carpeta del notebook. El objetivo es construir una narrativa descriptiva inicial, formular preguntas e hipótesis y responderlas con resúmenes numéricos y visualizaciones univariadas, bivariadas y multivariadas. El foco se limita estrictamente a las herramientas y conceptos vistos en clase: **Pandas** para carga/limpieza ligera y cálculo de resúmenes, **NumPy** cuando se requiera soporte numérico básico, **Matplotlib/Seaborn** para visualizaciones, y **estadística descriptiva** (medidas de tendencia central, dispersión, distribuciones, correlación y lectura de boxplots).

El dataset incluye variables numéricas habituales en audio (por ejemplo: *danceability*, *energy*, *loudness*, *speechiness*, *acousticness*, *instrumentalness*, *liveness*, *valence*, *tempo*, *duration_ms*) y metadatos como *year*, *artist* o *track*. Las preguntas guía son: **(P1)** ¿qué características de audio se asocian con la popularidad? **(P2)** ¿existen combinaciones de energía y valencia que distingan patrones de popularidad? **(P3)** ¿la duración, el tempo o la energía cambian su relación con la popularidad a lo largo de los años?

Hipótesis: **(H1)** *danceability* y *energy* se asocian positivamente con *popularity*; **(H2)** *valence* elevada (carácter “positivo” de la canción) se relaciona con mayor *popularity*; **(H3)** *loudness* y *tempo* muestran asociaciones moderadas con *popularity*; **(H4)** variables fuertemente estructurales entre sí (p. ej., *loudness* con *energy*) reflejan relaciones de producción más que de recepción del público. Para contrastarlas se emplean histogramas y boxplots (univariado/por grupos), gráficos de dispersión con codificación por color/tamaño (multivariado simple) y un mapa de correlaciones entre variables numéricas. También se cuantifican **valores ausentes**.

Los hallazgos esperados incluyen: asociaciones positivas de *danceability* y *energy* con *popularity*; relación entre *valence* y popularidad dependiente del período; y fuertes vínculos entre pares técnicos como *loudness–energy*. Esta lectura descriptiva prepara etapas futuras (no implementadas aquí) como modelos supervisados básicos, manteniendo el énfasis en **interpretabilidad**, **buenas prácticas visuales** y **trazabilidad** de acuerdo con lo visto en el curso.



## Preguntas e hipótesis

**P1.** ¿Características de audio como *danceability* o *energy* se asocian con la popularidad?  
**H1.** La relación con *popularity* es positiva.

**P2.** ¿El “carácter positivo” (*valence*) influye en la popularidad?  
**H2.** Asociación positiva, al menos moderada.

**P3.** ¿Tempo, duración o energía cambian su relación con la popularidad según el año?  
**H3.** Efectos diferenciados por período.

**P4.** ¿Existen relaciones fuertes entre variables técnicas (p. ej., *loudness–energy*)?  
**H4.** Correlación alta entre ciertas parejas técnicas.


## Carga de datos

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

df = pd.read_csv("songs_normalize.csv")
df.shape


## Exploración inicial y valores ausentes

In [ ]:

df.head()


In [ ]:

df.dtypes.to_frame("dtype").T


In [ ]:

na = df.isna().sum().sort_values(ascending=False).to_frame("faltantes")
na.T


## Resúmenes numéricos

In [ ]:

desc = df.describe().T
desc


## Selección de variables

In [ ]:

candidatas = [
    "popularity","danceability","energy","loudness","speechiness",
    "acousticness","instrumentalness","liveness","valence","tempo",
    "duration_ms","year"
]
num_cols = [c for c in candidatas if c in df.columns]
num_cols


## Visualizaciones univariadas

In [ ]:

cols_plot = [c for c in ["popularity","danceability","energy","valence","tempo"] if c in num_cols]
n = len(cols_plot)
rows = (n + 2) // 3
fig, axs = plt.subplots(rows, 3, figsize=(12, 4*rows))
axs = axs.ravel()
for i, c in enumerate(cols_plot):
    sns.histplot(df[c], kde=True, ax=axs[i])
    axs[i].set_title(c)
for j in range(i+1, rows*3):
    fig.delaxes(axs[j])
plt.tight_layout()
plt.show()


## Boxplots por grupos (año binned)

In [ ]:

if "year" in num_cols and "popularity" in num_cols:
    bins = pd.cut(df["year"], bins=[1999,2004,2009,2014,2019], labels=["2000-04","2005-09","2010-14","2015-19"])
    plt.figure(figsize=(8,4))
    sns.boxplot(x=bins, y=df["popularity"])
    plt.xlabel("Periodo")
    plt.ylabel("popularity")
    plt.title("Popularidad por período")
    plt.tight_layout()
    plt.show()


## Visualizaciones bivariadas y multivariadas

In [ ]:

if set(["danceability","energy","popularity"]).issubset(df.columns):
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df, x="danceability", y="energy", hue="popularity", alpha=0.7, legend=False)
    plt.title("Danceability vs Energy (color: popularity)")
    plt.tight_layout()
    plt.show()


In [ ]:

cands = [c for c in ["tempo","duration_ms"] if c in df.columns]
size_var = cands[0] if cands else None
if size_var and set(["valence","energy","popularity",size_var]).issubset(df.columns):
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=df, x="valence", y="energy", hue="popularity", size=size_var, sizes=(20,200), alpha=0.75)
    plt.title(f"Valence vs Energy (color: popularity, tamaño: {size_var})")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", borderaxespad=0.0)
    plt.tight_layout()
    plt.show()


## Mapa de correlaciones

In [ ]:

corr = df[num_cols].corr(numeric_only=True)
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap="RdBu_r", center=0)
plt.title("Correlaciones")
plt.tight_layout()
plt.show()
if "popularity" in corr.columns:
    corr["popularity"].sort_values(ascending=False)


## Detección visual de outliers

In [ ]:

vars_out = [c for c in ["popularity","danceability","energy","loudness","tempo"] if c in df.columns]
fig, axs = plt.subplots(1, len(vars_out), figsize=(4*len(vars_out), 4))
if len(vars_out) == 1:
    axs = [axs]
for i, c in enumerate(vars_out):
    sns.boxplot(y=df[c], ax=axs[i])
    axs[i].set_title(c)
plt.tight_layout()
plt.show()


## Interpretaciones


- No se presentan valores ausentes relevantes si la tabla de faltantes es cero; en caso contrario, se recomienda imputación simple según lo visto (media/mediana/moda) o eliminación si el volumen es marginal.
- Las distribuciones univariadas permiten observar sesgos y dispersión en variables como *popularity*, *danceability*, *energy*, *valence* y *tempo*.
- Los boxplots por período sugieren posibles cambios en la popularidad a lo largo del tiempo.
- Los dispersogramas multivariados apoyan **H1** y **H2** cuando se observa que mayor *danceability*/*energy* y *valence* se asocian con mayor *popularity*.
- El mapa de correlaciones ayuda a priorizar variables con mayor asociación con *popularity* y a identificar pares técnicos fuertemente relacionados.
- Los boxplots permiten una detección visual de outliers coherente con lo visto en clase.


## Conclusiones


1. **Popularidad y rasgos de audio.** Se observa, de forma descriptiva, que características como *danceability* y *energy* suelen asociarse positivamente con *popularity*. La relación de *valence* con la popularidad puede depender del período.
2. **Patrones por período.** Los resúmenes por grupos de años sugieren cambios en la distribución de *popularity*, útiles para hipótesis temporales.
3. **Relaciones técnicas.** Pares como *loudness–energy* muestran asociaciones altas, más vinculadas a producción que a recepción.
4. **Valores ausentes y outliers.** Se identifican faltantes si los hubiese y outliers de forma visual. Antes de modelar, se recomienda imputación simple y verificación de robustez.
5. **Siguientes pasos.** Profundizar en segmentaciones (por año o género si existe la columna), y eventualmente probar modelos supervisados básicos con validación, manteniendo las prácticas vistas.
